In [0]:
println("Hello,")
println(s"versions are: Java=${sys.props("java.version")} Scala=${scala.util.Properties.versionNumberString} Spark=${spark.version}")
println("enjoy the lab!")

Hello,
versions are: Java=17.0.10 Scala=2.13.6 Spark=3.4.2
enjoy the lab!


## Show tables with Scala and Spark

In [2]:
spark.catalog.listTables.show(false)

+----+-------------+---------+-----------+---------+-----------+
|name|catalog      |namespace|description|tableType|isTemporary|
+----+-------------+---------+-----------+---------+-----------+
|test|spark_catalog|[default]|null       |EXTERNAL |false      |
+----+-------------+---------+-----------+---------+-----------+



In [3]:
spark.sql("drop table if exists test")
val df = Seq(1,2,3).toDF("test")
df.write.option("path","/mnt/data/testTable").saveAsTable("default.test")

In [4]:
import org.apache.hadoop.fs.Path
val p = new Path("s3a://data/test/test.txt")
val fs = p.getFileSystem(spark.sparkContext.hadoopConfiguration)
//fs.mkdirs(p)
fs.exists(p)

true

In [5]:
spark.sql("drop table default.int_departures")

org.apache.spark.sql.catalyst.analysis.NoSuchTableException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `spark_catalog`.`default`.`int_departures` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.

In [6]:
spark.sql("select * from default.btl_distances where estarrivalairport = 'EDDR'").show

org.apache.spark.sql.AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `default`.`btl_distances` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'Filter ('estarrivalairport = EDDR)
   +- 'UnresolvedRelation [default, btl_distances], [], false


In [7]:
spark.table("default.int_departures").where($"estarrivalairport"==="LIMC").show

+-----------------------------+--------+-------------------------------+-----------------+------------------------------+-----------------------------+-------------------+--------------------------------+-------------------------------+----------+------+----------+--------+--------------------+
|arrivalAirportCandidatesCount|callsign|departureAirportCandidatesCount|estArrivalAirport|estArrivalAirportHorizDistance|estArrivalAirportVertDistance|estDepartureAirport|estDepartureAirportHorizDistance|estDepartureAirportVertDistance| firstSeen|icao24|  lastSeen|      dt|      dl_ts_captured|
+-----------------------------+--------+-------------------------------+-----------------+------------------------------+-----------------------------+-------------------+--------------------------------+-------------------------------+----------+------+----------+--------+--------------------+
|                            4|SSR07BM |                              0|             LIMC|                      

## Select data by using DataObjects configured in SmartDataLake

In [9]:
// import smartdatalake
import io.smartdatalake.config.SdlConfigObject.stringToDataObjectId
import io.smartdatalake.config.ConfigToolbox
import io.smartdatalake.workflow.dataobject._
import io.smartdatalake.workflow.ActionPipelineContext
import io.smartdatalake.workflow.action.SDLExecutionId
import io.smartdatalake.app.SmartDataLakeBuilderConfig
import io.smartdatalake.workflow.ExecutionPhase
implicit val ss = spark // make Spark session available implicitly

In [10]:
// read config from mounted directory
val (registry, globalConfig) = ConfigToolbox.loadAndParseConfig(Seq("/mnt/config"), Some(this.getClass.getClassLoader))
// Create the context used by SDL objects
implicit val context = ConfigToolbox.getDefaultActionPipelineContext(spark, registry)

In [11]:
// get a dataobject
val dataIntAirports = registry.get[DeltaLakeTableDataObject]("int-airports")
val dataIntDepartures = registry.get[DeltaLakeTableDataObject]("int-departures")

In [12]:
val df = dataIntAirports.getSparkDataFrame()

org.apache.spark.sql.AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `default`.`int_airports` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.;
'UnresolvedRelation [default, int_airports], [], false


In [13]:
import scala.collection.JavaConverters._
context.hadoopConf.getPropsWithPrefix("fs.s3a.aws").asScala
.foreach(println)

(.credentials.provider,org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider)


In [14]:
context.hadoopConf.set("fs.s3a.endpoint","http://s3:8888/")
context.hadoopConf.set("fs.s3a.secret.key","1234")
context.hadoopConf.set("fs.s3a.access.key","admin")
context.hadoopConf.set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
context.hadoopConf.set("fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
//dataIntAirports.prepare

Error: not found: value context (0)Error: not found: value context (61)Error: not found: value context (113)Error: not found: value context (166)Error: not found: value context (246)

In [15]:
import io.smartdatalake.definitions.Environment
import org.apache.hadoop.fs.Path

In [16]:
val fs = Environment.fileSystemFactory.getFileSystem(new Path("s3a://data/int-airports"), spark.sparkContext.hadoopConfiguration)

In [17]:
val p = new Path("s3a://data/int-airports")
val fs = p.getFileSystem(context.hadoopConf)
context.hadoopConf.getPropsWithPrefix("fs.s3a.bucket").asScala
.foreach(println)
//fs.exists(p)

(.data.secret.key,1234)
(.data.access.key,admin)
(.data.impl,org.apache.hadoop.fs.s3a.S3AFileSystem)
(.data.endpoint,http://s3:8888/)


In [18]:
fs.exists(p)

java.nio.file.AccessDeniedException: s3a://data/int-airports: org.apache.hadoop.fs.s3a.auth.NoAuthWithAWSException: No AWS Credentials provided by TemporaryAWSCredentialsProvider SimpleAWSCredentialsProvider EnvironmentVariableCredentialsProvider IAMInstanceCredentialsProvider : com.amazonaws.SdkClientException: Unable to load AWS credentials from environment variables (AWS_ACCESS_KEY_ID (or AWS_ACCESS_KEY) and AWS_SECRET_KEY (or AWS_SECRET_ACCESS_KEY))

In [19]:
import org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider
val scp = new SimpleAWSCredentialsProvider(p.toUri, spark.sparkContext.hadoopConfiguration)
val c = scp.getCredentials
print(c.getAWSSecretKey)

1234

## Historization of airport data

In [21]:
dataIntDepartures.dropTable

Start Action historize-airports

In [23]:
dataIntAirports.getSparkDataFrame().printSchema

root
 |-- ident: string (nullable = true)
 |-- name: string (nullable = true)
 |-- latitude_deg: string (nullable = true)
 |-- longitude_deg: string (nullable = true)
 |-- dl_ts_captured: timestamp (nullable = true)
 |-- dl_ts_delimited: timestamp (nullable = true)


In [24]:
dataIntAirports.getSparkDataFrame().orderBy($"ident",$"dl_ts_captured").show

+-----+--------------------+------------------+-------------------+--------------------+-------------------+
|ident|                name|      latitude_deg|      longitude_deg|      dl_ts_captured|    dl_ts_delimited|
+-----+--------------------+------------------+-------------------+--------------------+-------------------+
|  00A|   Total RF Heliport|         40.070985|         -74.933689|2023-10-02 12:10:...|9999-12-31 00:00:00|
| 00AA|Aero B Ranch Airport|         38.704022|        -101.473911|2023-10-02 12:10:...|9999-12-31 00:00:00|
| 00AK|        Lowell Field|         59.947733|        -151.692524|2023-10-02 12:10:...|9999-12-31 00:00:00|
| 00AL|        Epps Airpark| 34.86479949951172| -86.77030181884766|2023-10-02 12:10:...|9999-12-31 00:00:00|
| 00AN|Katmai Lodge Airport|         59.093287|        -156.456699|2023-10-02 12:10:...|9999-12-31 00:00:00|
| 00AR|Newport Hospital ...|           35.6087|         -91.254898|2023-10-02 12:10:...|9999-12-31 00:00:00|
| 00AS|      Fulton

In [25]:
dataIntAirports.dropTable

Delete all files in data/stg-airport and copy the historical result.csv from the folder data-fallback-download/stg-airport into the folder data/stg-aiport.
Now start the action historize-airports. Afterwards, start actions download-airports and historize-airports to download fresh data and build up the airport history.

In [27]:
dataIntAirports.getSparkDataFrame()
  .groupBy($"ident").count
  .orderBy($"count".desc)
  .show

+-------+-----+
|  ident|count|
+-------+-----+
|   0OH7|    2|
|   MTPX|    2|
|   1CL8|    2|
|   2LA2|    2|
|   36FA|    2|
|   8LL0|    2|
|AF-0006|    2|
|AU-0083|    2|
|CA-0259|    2|
|   ESSX|    2|
|   FBGD|    2|
|   GE25|    2|
|HU-0043|    2|
|   K1O4|    2|
|   KAAT|    2|
|   KCKA|    2|
|KZ-0016|    2|
|   LA65|    2|
|   LETC|    2|
|   LOGP|    2|
+-------+-----+


In [28]:
dataIntAirports.getSparkDataFrame()
  .where($"ident"==="CDV3")
  .show(false)

+-----+-------------------------------------------------+-------------+--------------+--------------------------+--------------------------+
|ident|name                                             |latitude_deg |longitude_deg |dl_ts_captured            |dl_ts_delimited           |
+-----+-------------------------------------------------+-------------+--------------+--------------------------+--------------------------+
|CDV3 |Charlottetown (Queen Elizabeth Hospital) Heliport|46.2554925916|-63.0988866091|2023-10-02 12:14:02.575663|2023-10-02 12:14:28.086627|
|CDV3 |Charlottetown (Queen Elizabeth Hospital) Heliport|46.255493    |-63.098887    |2023-10-02 12:14:28.087627|9999-12-31 00:00:00       |
+-----+-------------------------------------------------+-------------+--------------+--------------------------+--------------------------+


## Deduplication of flight data

In [30]:
val dataIntDepartures = registry.get[DeltaLakeTableDataObject]("int-departures")
dataIntDepartures.dropTable

Start Action deduplicate-departures

In [32]:
dataIntDepartures.getSparkDataFrame().printSchema

root
 |-- arrivalAirportCandidatesCount: long (nullable = true)
 |-- callsign: string (nullable = true)
 |-- departureAirportCandidatesCount: long (nullable = true)
 |-- estArrivalAirport: string (nullable = true)
 |-- estArrivalAirportHorizDistance: long (nullable = true)
 |-- estArrivalAirportVertDistance: long (nullable = true)
 |-- estDepartureAirport: string (nullable = true)
 |-- estDepartureAirportHorizDistance: long (nullable = true)
 |-- estDepartureAirportVertDistance: long (nullable = true)
 |-- firstSeen: long (nullable = true)
 |-- icao24: string (nullable = true)
 |-- lastSeen: long (nullable = true)
 |-- dt: string (nullable = true)
 |-- dl_ts_captured: timestamp (nullable = true)


In [33]:
dataIntDepartures.getSparkDataFrame()
  .groupBy($"icao24", $"estdepartureairport", $"dt")
  .count
  .orderBy($"count".desc)
  .show

+------+-------------------+--------+-----+
|icao24|estdepartureairport|      dt|count|
+------+-------------------+--------+-----+
|4b43ab|               LSZB|20210829|    3|
|4b4b8d|               LSZB|20210829|    3|
|4b1b13|               LSZB|20210829|    2|
|4b4445|               LSZB|20210829|    2|
|4b4442|               LSZB|20210829|    1|
|4b43ab|               LSZB|20210830|    1|
|346603|               LSZB|20210829|    1|
|4b4449|               LSZB|20210829|    1|
|4b0f70|               LSZB|20210830|    1|
|49d283|               LSZB|20210829|    1|
|4b1a01|               LSZB|20210829|    1|
|4b4445|               LSZB|20210830|    1|
|4b51fa|               LSZB|20210829|    1|
|aa0da1|               LSZB|20210829|    1|
|4d02d7|               LSZB|20210829|    1|
|4b1b12|               LSZB|20210829|    1|
|494108|               LSZB|20210829|    1|
|4b3049|               LSZB|20210829|    1|
|4b1f2f|               LSZB|20210830|    1|
|44046b|               LSZB|2021